# BreastNet++ - Efficient Breast Cancer Classifier (Optimized for GTX 1650)

In [ ]:
!pip install -q efficientnet_pytorch albumentations timm

In [ ]:
import os # Imports the 'os' module for interacting with the operating system, like file paths and directories.
import torch # Imports the main PyTorch library.
import torch.nn as nn # Imports the neural network module from PyTorch.
from torch.utils.data import DataLoader, Dataset # Imports DataLoader for batching data and Dataset for creating custom datasets.
from efficientnet_pytorch import EfficientNet # Imports the EfficientNet model from the 'efficientnet_pytorch' library.
import albumentations as A # Imports the Albumentations library for image augmentation.
from albumentations.pytorch import ToTensorV2 # Imports ToTensorV2 from Albumentations for converting images to PyTorch tensors.
import cv2 # Imports the OpenCV library for image processing.
from tqdm import tqdm # Imports tqdm for displaying progress bars during iterations.

# CBAM Attention Module
class CBAM(nn.Module): # Defines the Convolutional Block Attention Module (CBAM) as a PyTorch module.
    def __init__(self, channels, reduction=16, kernel_size=7): # Constructor takes input 'channels', 'reduction' ratio for channel attention, and 'kernel_size' for spatial attention.
        super(CBAM, self).__init__() # Calls the constructor of the parent class (nn.Module).
        # Channel Attention Module (CAM) branch.
        self.channel_attention = nn.Sequential(
            nn.AdaptiveAvgPool2d(1), # Applies global average pooling to the input features, reducing spatial dimensions to 1x1.
            nn.Conv2d(channels, channels // reduction, 1, bias=False), # First 1x1 convolution for channel attention, reducing dimensions.
            nn.ReLU(), # ReLU activation function.
            nn.Conv2d(channels // reduction, channels, 1, bias=False), # Second 1x1 convolution, expanding dimensions back to original channels.
            nn.Sigmoid() # Sigmoid activation to produce channel attention weights between 0 and 1.
        )
        # Spatial Attention Module (SAM) branch.
        self.spatial_attention = nn.Sequential(
            # A convolutional layer operating on a concatenated feature map (avg pool + max pool of channels).
            nn.Conv2d(2, 1, kernel_size, padding=kernel_size // 2, bias=False), 
            nn.Sigmoid() # Sigmoid activation to produce spatial attention weights between 0 and 1.
        )

    def forward(self, x): # Defines the forward pass for the CBAM module.
        ca = self.channel_attention(x) # Computes channel attention weights.
        x = x * ca # Multiplies the input feature map 'x' by the channel attention weights, broadcasting across spatial dimensions.
        max_out, _ = torch.max(x, dim=1, keepdim=True) # Computes the maximum value across the channel dimension, keeping the dimension.
        mean_out = torch.mean(x, dim=1, keepdim=True) # Computes the mean value across the channel dimension, keeping the dimension.
        sa_input = torch.cat([max_out, mean_out], dim=1) # Concatenates the max-pooled and mean-pooled features along the channel dimension for spatial attention.
        sa = self.spatial_attention(sa_input) # Computes spatial attention weights.
        x = x * sa # Multiplies the channel-attended feature map 'x' by the spatial attention weights, broadcasting across channel dimensions.
        return x # Returns the feature map with both channel and spatial attention applied.

# EfficientNet-B0 + CBAM model
class BreastNetPP(nn.Module): # Defines the main BreastNetPP model, which integrates EfficientNet-B0 and CBAM.
    def __init__(self): # Constructor for the BreastNetPP model.
        super(BreastNetPP, self).__init__() # Calls the parent class constructor.
        self.backbone = EfficientNet.from_pretrained('efficientnet-b0') # Initializes EfficientNet-B0 as the backbone, pre-trained on ImageNet.
        self.cbam = CBAM(1280) # Initializes the CBAM module. EfficientNet-B0's last feature map has 1280 channels.
        self.pool = nn.AdaptiveAvgPool2d(1) # Global Adaptive Average Pooling to reduce spatial dimensions to 1x1.
        self.dropout1 = nn.Dropout(0.5) # First dropout layer with a 50% dropout rate.
        self.fc1 = nn.Linear(1280, 128) # First fully connected layer: input 1280 features (from pooled backbone+CBAM), output 128 features.
        self.relu = nn.ReLU() # ReLU activation after the first fully connected layer.
        self.dropout2 = nn.Dropout(0.3) # Second dropout layer with a 30% dropout rate.
        self.fc2 = nn.Linear(128, 1) # Output fully connected layer: maps 128 features to 1 output feature (for binary classification).
        self.sigmoid = nn.Sigmoid() # Sigmoid activation function applied to the final output, squishing values to [0, 1] for binary probability.

    def forward(self, x): # Defines the forward pass of the BreastNetPP model.
        x = self.backbone.extract_features(x) # Extracts features using the EfficientNet backbone.
        x = self.cbam(x) # Applies the CBAM attention module to the extracted features.
        x = self.pool(x) # Applies global adaptive average pooling.
        x = x.view(x.size(0), -1) # Flattens the output from the pooling layer, keeping the batch dimension.
        x = self.dropout1(x) # Applies the first dropout.
        x = self.relu(self.fc1(x)) # Passes through the first fully connected layer and ReLU activation.
        x = self.dropout2(x) # Applies the second dropout.
        return self.sigmoid(self.fc2(x)) # Passes through the final fully connected layer and applies sigmoid activation before returning the output.

d:\Anaconda\envs\nuscenes_env\lib\site-packages\albumentations\__init__.py:13: UserWarning: A new version of Albumentations is available: 2.0.8 (you have 1.4.18). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [ ]:
class CustomDataset(Dataset): # Defines a custom PyTorch Dataset class, inheriting from torch.utils.data.Dataset.
    def __init__(self, root_dir, transform=None): # Constructor for the CustomDataset.
        self.root_dir = root_dir # Stores the root directory where the dataset images are located (e.g., 'Preprocessed/train').
        self.transform = transform # Stores the image transformations (e.g., augmentations, normalization) to be applied.
        self.classes = ['benign', 'malignant'] # Defines the class names, assuming 'benign' maps to label 0 and 'malignant' to label 1.
        self.image_paths = [] # Initializes an empty list to store the full paths to all images.
        self.labels = [] # Initializes an empty list to store the corresponding numerical labels for each image.

        # Iterates through each class (e.g., 'benign', 'malignant') and assigns an integer label (0, 1).
        for label, cls in enumerate(self.classes):
            cls_path = os.path.join(root_dir, cls) # Constructs the full path to the current class directory (e.g., 'root_dir/benign').
            for img_name in os.listdir(cls_path): # Iterates through each image file name within the current class directory.
                self.image_paths.append(os.path.join(cls_path, img_name)) # Appends the full path of the image to the 'image_paths' list.
                self.labels.append(label) # Appends the corresponding numerical label (0 or 1) to the 'labels' list.

    def __len__(self): # Defines the __len__ method, which is required for PyTorch Datasets.
        return len(self.image_paths) # Returns the total number of images in the dataset.

    def __getitem__(self, idx): # Defines the __getitem__ method, which is required for PyTorch Datasets. It retrieves an item (image and label) by its index.
        image = cv2.imread(self.image_paths[idx]) # Reads the image from the stored path at the given index using OpenCV (reads as BGR by default).
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # Converts the image from BGR (OpenCV's default) to RGB, which is typically expected by deep learning models.
        label = self.labels[idx] # Retrieves the numerical label for the image at the given index.

        if self.transform: # Checks if any transformations were provided to the dataset.
            # If transformations are present (assuming Albumentations transform), applies them.
            # Albumentations expects a dictionary input {'image': image} and returns a dictionary.
            image = self.transform(image=image)['image'] 

        # Returns the transformed image tensor and the label as a PyTorch tensor (float32 for consistency, especially with BCEWithLogitsLoss).
        return image, torch.tensor(label, dtype=torch.float32)

In [ ]:
train_transform = A.Compose([ # Defines a composition of image transformations to be applied to the training dataset using Albumentations.
    A.Resize(160, 160), # Resizes the input image to 160x160 pixels. This is the desired input size for the model.
    A.HorizontalFlip(p=0.5), # Randomly flips the image horizontally with a probability of 0.5 (50% chance). This is a common data augmentation technique.
    A.RandomBrightnessContrast(p=0.2), # Randomly adjusts the brightness and contrast of the image with a probability of 0.2. Another data augmentation.
    # Normalizes the pixel values of the image.
    # 'mean' and 'std' are typically derived from a large dataset like ImageNet to make the data compatible with pre-trained models.
    # The normalization converts pixel values from [0, 255] to a distribution with mean 0 and standard deviation 1.
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
    ToTensorV2() # Converts the NumPy array image (output of Albumentations) into a PyTorch tensor.
])

val_transform = A.Compose([ # Defines a composition of image transformations for the validation (and typically test) dataset.
    A.Resize(160, 160), # Resizes the input image to 160x160 pixels. This should match the training size.
    # Normalizes the pixel values using the same mean and standard deviation as the training set.
    # No data augmentation (like flips or brightness changes) is typically applied to validation/test sets to ensure consistent evaluation.
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)), 
    ToTensorV2() # Converts the NumPy array image to a PyTorch tensor.
])

In [ ]:
train_dataset = CustomDataset("./Preprocessed/train", transform=train_transform) # Creates an instance of CustomDataset for the training data.
# It points to the './Preprocessed/train' directory as the root and applies the 'train_transform' (which includes augmentations).
val_dataset = CustomDataset("./Preprocessed/val", transform=val_transform) # Creates an instance of CustomDataset for the validation data.
# It points to the './Preprocessed/val' directory and applies the 'val_transform' (which includes only resizing and normalization).
test_dataset = CustomDataset("./Preprocessed/test", transform=val_transform) # Creates an instance of CustomDataset for the test data.
# It points to the './Preprocessed/test' directory and also applies the 'val_transform'.

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True) # Creates a DataLoader for the training dataset.
# 'batch_size=8' means data will be loaded in batches of 8 images.
# 'shuffle=True' means the data will be reshuffled at every epoch, which is good practice for training to prevent the model from learning the order of samples.
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False) # Creates a DataLoader for the validation dataset.
# 'shuffle=False' because the order of validation samples doesn't need to be randomized, and it allows for consistent evaluation.
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False) # Creates a DataLoader for the test dataset.
# 'shuffle=False' for the same reason as the validation set: consistent and reproducible evaluation.

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Detects if a CUDA-enabled GPU is available. If yes, sets 'device' to 'cuda'; otherwise, sets it to 'cpu'. This determines where the model and tensors will reside and computations will be performed.
model = BreastNetPP().to(device) # Creates an instance of the BreastNetPP model and immediately moves it to the detected device (GPU or CPU). All model parameters will now be on this device.
criterion = nn.BCELoss() # Defines the loss function for the model. BCELoss (Binary Cross-Entropy Loss) is suitable for binary classification tasks where the model's output is a probability (after a sigmoid activation).
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4) # Initializes the optimizer. Adam is an adaptive learning rate optimization algorithm.
# 'model.parameters()' tells the optimizer which parameters (weights and biases) of the model it needs to update during training.
# 'lr=1e-4' sets the initial learning rate to 0.0001.

Loaded pretrained weights for efficientnet-b0


In [ ]:
def train(model, loader, optimizer, criterion): # Defines the training function for one epoch.
    model.train() # Sets the model to training mode. This enables features like Dropout and Batch Normalization updates.
    running_loss = 0 # Initializes a variable to accumulate the total loss over the epoch.
    loop = tqdm(loader, desc="Training", leave=False) # Creates a TQDM progress bar for the training loader, showing "Training" description and not leaving the bar after completion.
    for imgs, labels in loop: # Iterates through each batch of images and labels from the data loader.
        imgs, labels = imgs.to(device), labels.to(device).unsqueeze(1) # Moves image tensors and label tensors to the specified device (GPU/CPU).
        # 'labels.unsqueeze(1)' changes the shape of labels from (batch_size) to (batch_size, 1), which is often required by BCELoss for consistency with model output.
        optimizer.zero_grad() # Clears the gradients of all optimized tensors. This is crucial before computing new gradients for the current batch.
        outputs = model(imgs) # Performs a forward pass: passes the images through the model to get predictions (logits or probabilities, depending on the model's last layer).
        loss = criterion(outputs, labels) # Calculates the loss between the model's outputs and the true labels using the defined criterion (e.g., BCELoss).
        loss.backward() # Performs a backward pass: computes the gradients of the loss with respect to all learnable parameters of the model.
        optimizer.step() # Updates the model's parameters using the computed gradients and the chosen optimizer's update rule.
        running_loss += loss.item() # Adds the current batch's loss (as a standard Python number) to the running total loss.
        loop.set_postfix(loss=loss.item()) # Updates the TQDM progress bar with the current batch's loss.
    return running_loss / len(loader) # Returns the average loss for the entire epoch.

def evaluate(model, loader, criterion): # Defines the evaluation function for one epoch (validation or testing).
    model.eval() # Sets the model to evaluation mode. This disables Dropout and sets Batch Normalization to use running means and variances, ensuring deterministic output.
    total, correct = 0, 0 # Initializes counters for total samples and correctly predicted samples.
    running_loss = 0 # Initializes a variable to accumulate the total loss during evaluation.
    loop = tqdm(loader, desc="Evaluating", leave=False) # Creates a TQDM progress bar for the evaluation loader.
    with torch.no_grad(): # Disables gradient calculations for all operations within this block. This saves memory and speeds up computation during inference.
        for imgs, labels in loop: # Iterates through each batch of images and labels from the data loader.
            imgs, labels = imgs.to(device), labels.to(device).unsqueeze(1) # Moves images and labels to the specified device, unsqueezing labels as in training.
            outputs = model(imgs) # Performs a forward pass to get model predictions.
            loss = criterion(outputs, labels) # Calculates the loss for the current batch.
            running_loss += loss.item() # Accumulates the loss.
            preds = (outputs > 0.5).float() # Converts model probabilities (outputs from sigmoid) into binary predictions (0 or 1) using a 0.5 threshold.
            correct += (preds == labels).sum().item() # Compares predictions with true labels, sums up the correct ones, and adds to the total correct count. '.item()' converts a single-element tensor to a Python number.
            total += labels.size(0) # Adds the number of samples in the current batch to the total count.
            loop.set_postfix(loss=loss.item()) # Updates the TQDM progress bar with the current batch's loss.
    accuracy = correct / total # Calculates the overall accuracy for the evaluation epoch.
    return running_loss / len(loader), accuracy # Returns the average loss and the accuracy for the entire evaluation epoch.

In [ ]:
save_dir = "saved_models" # Defines the directory name where model checkpoints will be saved.
os.makedirs(save_dir, exist_ok=True) # Creates the specified directory if it doesn't already exist. 'exist_ok=True' prevents an error if the directory already exists.

num_epochs = 5 # Sets the total number of training epochs.
for epoch in range(num_epochs): # Starts a loop that iterates for the specified number of epochs.
    print(f"\n🔁 Epoch {epoch+1}/{num_epochs}") # Prints the current epoch number (1-indexed for user readability).
    torch.cuda.empty_cache() # Clears unused cached memory from the GPU. This can help prevent 'out of memory' errors during training, especially with large models or data.
    train_loss = train(model, train_loader, optimizer, criterion) # Calls the 'train' function to perform one full training pass over the training data.
                                                                # It returns the average training loss for that epoch.
    val_loss, val_acc = evaluate(model, val_loader, criterion) # Calls the 'evaluate' function to assess the model's performance on the validation data.
                                                              # It returns the average validation loss and validation accuracy.

    print(f"✅ Epoch {epoch+1} | Train Loss: {train_loss:.4f} | " # Prints a summary of the epoch's results.
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}") # Formats the loss and accuracy to four decimal places.

    checkpoint = { # Creates a dictionary to store the current state of the training process, known as a checkpoint.
        'epoch': epoch + 1, # Stores the current epoch number.
        'model_state_dict': model.state_dict(), # Stores the learned parameters (weights and biases) of the model. This is essential for resuming training or loading the trained model later.
        'optimizer_state_dict': optimizer.state_dict(), # Stores the current state of the optimizer (e.g., learning rate, momentum buffers for Adam). This is important for resuming training from this point.
        'val_loss': val_loss, # Stores the validation loss achieved in this epoch.
        'val_accuracy': val_acc # Stores the validation accuracy achieved in this epoch.
    }
    # Saves the checkpoint dictionary to a file. The filename includes the model's code name (brestnetPP) and the epoch number, 
    # allowing you to track progress and load specific versions of the model.
    torch.save(checkpoint, f"{save_dir}/brestnetpp_epoch_{epoch+1}.pth")

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report # Imports various metrics functions from scikit-learn for model evaluation.
import torch # Imports the main PyTorch library.
import numpy as np # Imports NumPy for numerical operations, especially for array manipulation.

def test_model(model, test_loader, checkpoint_path): # Defines a function to test the model's performance on a test dataset.
    # Load checkpoint
    checkpoint = torch.load(checkpoint_path, map_location=device) # Loads the model checkpoint from the specified 'checkpoint_path'. 'map_location=device' ensures the checkpoint is loaded onto the correct device (CPU or GPU).
    model.load_state_dict(checkpoint['model_state_dict']) # Loads the saved state dictionary (trained weights and biases) into the 'model' instance.
    model.to(device) # Moves the entire model to the specified device (e.g., GPU if available).
    model.eval() # Sets the model to evaluation mode. This is crucial for inference as it disables dropout layers and sets batch normalization layers to use their learned population statistics rather than batch statistics.

    all_preds = [] # Initializes an empty list to store all the predicted class labels (0 or 1).
    all_labels = [] # Initializes an empty list to store all the true class labels.

    with torch.no_grad(): # A context manager that disables gradient calculation. This significantly reduces memory consumption and speeds up computation during inference as no gradients are needed for backpropagation.
        for imgs, labels in test_loader: # Iterates through each batch of images and their corresponding true labels from the 'test_loader'.
            imgs = imgs.to(device) # Moves the input image tensor to the specified device.
            labels = labels.to(device).unsqueeze(1) # Moves the true label tensor to the device and reshapes it to (batch_size, 1) to match the expected output shape from the model for binary classification.
            outputs = model(imgs) # Performs a forward pass: feeds the images through the model to obtain raw predictions (logits).
            preds = (outputs > 0.5).float() # Converts the model's continuous output probabilities (assuming a sigmoid output from the brestnetPP model) into discrete binary predictions (0 or 1) using a threshold of 0.5.
            all_preds.extend(preds.cpu().numpy()) # Moves the predicted tensor to the CPU, converts it to a NumPy array, and extends the 'all_preds' list with these predictions.
            all_labels.extend(labels.cpu().numpy()) # Moves the true label tensor to the CPU, converts it to a NumPy array, and extends the 'all_labels' list with these true labels.

    # Flatten predictions and labels
    y_pred = np.array(all_preds).flatten() # Converts the list of all predictions into a single, one-dimensional NumPy array.
    y_true = np.array(all_labels).flatten() # Converts the list of all true labels into a single, one-dimensional NumPy array.

    # Evaluation Metrics
    acc = accuracy_score(y_true, y_pred) # Calculates the overall accuracy: the proportion of correctly classified instances.
    prec = precision_score(y_true, y_pred, zero_division=0) # Calculates the precision: the proportion of positive identifications that were actually correct. 'zero_division=0' handles cases where there are no true positives + false positives, preventing a warning and returning 0.
    rec = recall_score(y_true, y_pred, zero_division=0) # Calculates the recall: the proportion of actual positives that were identified correctly.
    f1 = f1_score(y_true, y_pred, zero_division=0) # Calculates the F1-score: the harmonic mean of precision and recall, providing a single metric that balances both.

    print("📊 Evaluation Results:") # Prints a header for the evaluation results.
    print(f"✅ Accuracy   : {acc:.4f}") # Prints the calculated accuracy, formatted to four decimal places.
    print(f"✅ Precision : {prec:.4f}") # Prints the calculated precision, formatted to four decimal places.
    print(f"✅ Recall    : {rec:.4f}") # Prints the calculated recall, formatted to four decimal decades.
    print(f"✅ F1-Score  : {f1:.4f}") # Prints the calculated F1-score, formatted to four decimal places.
    print("\nClassification Report:") # Prints a header for the classification report.
    # Prints a detailed text report showing the main classification metrics (precision, recall, f1-score) for each class, along with support.
    # 'target_names' provides human-readable names for the classes.
    print(classification_report(y_true, y_pred, target_names=["Benign", "Malignant"]))
    print("Confusion Matrix:") # Prints a header for the confusion matrix.
    # Prints the confusion matrix, which is a table used to describe the performance of a classification model on a set of test data for which the true values are known.
    print(confusion_matrix(y_true, y_pred))

In [ ]:
# Calls the 'test_model' function to evaluate the 'model' using the 'test_loader'.
# It specifies the path to a saved model checkpoint: "saved_models/breastnetpp_epoch_5.pth".
# This means the evaluation will be performed on the 'brestnetPP' model's state as it was at the end of epoch 5.
test_model(model, test_loader, "saved_models/breastnetpp_epoch_5.pth")

📊 Evaluation Results:
✅ Accuracy  : 0.9072
✅ Precision : 0.8201
✅ Recall    : 0.8625
✅ F1-Score  : 0.8407

Classification Report:
              precision    recall  f1-score   support

      Benign       0.94      0.92      0.93     39748
   Malignant       0.82      0.86      0.84     15758

    accuracy                           0.91     55506
   macro avg       0.88      0.89      0.89     55506
weighted avg       0.91      0.91      0.91     55506

Confusion Matrix:
[[36766  2982]
 [ 2167 13591]]
